## 1.  Custom Loss Function

## Points to note before creating custom loss:

* Need to store the thresholds while saving & using the h5 models again with custom loss

* In such cases use subclass method compared to simpler way of pythonic functions

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
(xtrain, ytrain), (xtest, ytest) = tf.keras.datasets.boston_housing.load_data()
scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)
xtrain, xval = xtrain[:350], xtrain[350:]
ytrain, yval = ytrain[:350], ytrain[350:]

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = [13]),
    tf.keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dense(1),
    ])
model.compile(loss = 'mae', metrics = ['mae'], optimizer = 'Adam')
history = model.fit(xtrain, ytrain, epochs = 100, validation_data = (xval, yval))

In [ ]:
plt.plot(np.arange(100), history.history["val_mae"], label = 'val_error')
plt.plot(np.arange(100), history.history["mae"], label = 'training_error')
plt.legend()

In [ ]:
model.evaluate(xtest, ytest)

In [ ]:
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold = 1.0, **kwargs):
        ''' takes into account of standard hyperparms, std aggregation of loss (sum_over_batch_size)'''
        self.threshold = threshold
        super().__init__(**kwargs)
    
    def call(self, y_true, y_pred):
        ''' compute loss and returns '''
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_error = tf.square(error)/2
        linear_error = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_error, linear_error)

    def get_config(self):
        ''' returns hyperparams name to its value '''
        base_config = super().get_config()
        return {**base_config, 'threshold' : self.threshold}

In [ ]:
model.compile(loss = HuberLoss(2.), optimizer = 'Adam')
history = model.fit(xtrain, ytrain, epochs = 100, validation_data = (xval, yval))

In [ ]:
plt.plot(np.arange(100), history.history["val_loss"], label = 'val_error')
plt.plot(np.arange(100), history.history["loss"], label = 'training_error')
plt.legend()

In [ ]:
model.evaluate(xtest, ytest)

In [ ]:
## Saving Model
tf.keras.models.save_model(model, "my_model_custom_loss_class.h5")

In [3]:
# model.compile(loss = HuberLoss(2.), optimizer = 'Adam')
model_from_saved = tf.keras.models.load_model("my_model_custom_loss_class.h5", custom_objects={'HuberLoss' : HuberLoss()}, compile=False)

NameError: name 'HuberLoss' is not defined

In [ ]:
model_from_saved.compile(loss = HuberLoss(2.), optimizer = 'Adam')

model_from_saved.evaluate(xtest, ytest)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name='3_layer_mlp')

# Useless custom loss here
def custom_loss(y_true, y_pred):
    return keras.backend.mean(keras.backend.square(y_true - y_pred), axis=-1)

model.save("model", save_format='tf')
model.compile(loss=custom_loss, optimizer=keras.optimizers.RMSprop())
# Here comes the bug (no bug)
new_model = keras.models.load_model('model', custom_objects={'loss': custom_loss})

### 2. Custom Activations, Regularizers, Initializers

#### Points to note:
* Use always tf functionality to maximise the performance

* you must implement the call() method for losses, layers (including activation
functions), and models, or the __call__() method for regularizers, initializers,
and constraints.

In [9]:
def softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

def glorot_initializer(shape, dtype = tf.float32):
    stddev = tf.sqrt(2./(shape[0] + shape[1]))
    return tf.random.normal(shape, mean=0, stddev=stddev, dtype=dtype)

def l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.1 * weights))

def relu_constraint(weights):
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [5]:
(xtrain, ytrain), (xtest, ytest) = tf.keras.datasets.boston_housing.load_data()
scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)
xtrain, xval = xtrain[:350], xtrain[350:]
ytrain, yval = ytrain[:350], ytrain[350:]

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = [13]),
    tf.keras.layers.Dense(10, activation=softplus, kernel_initializer=glorot_initializer,
                         kernel_regularizer=l1_regularizer, kernel_constraint=relu_constraint),
    tf.keras.layers.Dense(1),
    ])
model.compile(loss = 'mae', metrics = ['mae'], optimizer = 'Adam')
history = model.fit(xtrain, ytrain, epochs = 100, validation_data = (xval, yval))

Train on 350 samples, validate on 54 samples
Epoch 1/100
350/350 [==============================] - 1s 2ms/sample - loss: 24.9289 - mae: 23.0378 - val_loss: 24.8066 - val_mae: 23.0778
Epoch 2/100
350/350 [==============================] - 0s 126us/sample - loss: 24.6206 - mae: 22.9163 - val_loss: 24.6217 - val_mae: 22.9471
Epoch 3/100
350/350 [==============================] - 0s 140us/sample - loss: 24.4236 - mae: 22.7728 - val_loss: 24.4392 - val_mae: 22.8169
Epoch 4/100
350/350 [==============================] - 0s 134us/sample - loss: 24.2302 - mae: 22.6307 - val_loss: 24.2590 - val_mae: 22.6872
Epoch 5/100
350/350 [==============================] - 0s 137us/sample - loss: 24.0379 - mae: 22.4883 - val_loss: 24.0825 - val_mae: 22.5594
Epoch 6/100
350/350 [==============================] - 0s 146us/sample - loss: 23.8508 - mae: 22.3484 - val_loss: 23.9094 - val_mae: 22.4316
Epoch 7/100
350/350 [==============================] - 0s 143us/sample - loss: 23.6683 - mae: 22.2099 - val_los

In [15]:
## Use subclass approach if you want to save l1 regularizer factor along with model
class l1_regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor, **kwargs):
        self.factor = factor

    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    
    def get_config(self):
        return {'factor' : self.factor}

### 3. Custom Layers


In [3]:
## this layer creates exponential of inputs
exponential_layer = tf.keras.layers.Lambda(lambda x : tf.exp(x))